In [1]:
from typing import List
import torch
from torch import nn, optim
import numpy as np
#from pinn import PINN, IPINN

In [2]:
from typing import List, Callable, Optional

In [3]:
from plotly import graph_objects as go, io as pio
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
torch.set_default_dtype(torch.double)
torch.set_default_device(torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

In [5]:
class MLP(nn.Module):
    """Multi-Layer Perceptron (MLP) module."""

    def __init__(self, layer_size: List[int], activation: nn.Module = nn.Tanh()):
        super().__init__()

        self.linear = nn.ModuleList()
        for i in range(1, len(layer_size)):
            self.linear.append(nn.Linear(layer_size[i - 1], layer_size[i]))
        self.activation = activation

    def forward(self, x):
        for i, item in enumerate(self.linear[:-1]):
            x = self.activation(item(x))
        x = self.linear[-1](x)
        return x

In [6]:
class IPINN:
    """Physics-Informed Neural Network (PINN) class."""

    def __init__(self, layer_size: List[int], activation: nn.Module = nn.Tanh()):
        self.mlp = MLP(layer_size, activation)
        self.optimizer: Optional[optim.Optimizer] = None
        self.loss = nn.MSELoss()
        self.loss_history: List[float] = []
        self.param_history: List[List[float]] = []

    def compile(
        self,
        ftns: List[
            Callable[[torch.Tensor, torch.Tensor, List[torch.Tensor]], torch.Tensor]
        ],
        pts: List[torch.Tensor],
        observation_pts: List[torch.Tensor],
        observation_val: List[torch.Tensor],
        parameter: List[torch.Tensor],
    ):
        self.ftns = ftns
        self.pts = pts
        self.observation_pts = observation_pts
        self.observation_val = observation_val
        parameter = list(map(lambda param: param.requires_grad_(True), parameter))
        self.parameter = parameter

        self.optimizer = optim.Adam(
            list(self.mlp.parameters()) + list(self.parameter), lr=1e-3
        )
        if not len(self.ftns) == len(self.pts):
            raise ValueError(f"Arguments `ftns` and `pts` must have the same length.")
        if not len(self.observation_val) == len(self.observation_pts):
            raise ValueError(
                f"Arguments `observation_val` and `observation_pts` must have the same length."
            )

    def train(self, epochs: int, loss_weights: Optional[List[float]] = None):
        self.mlp.train()
        if loss_weights is None:
            loss_weights = [1.0 for _ in self.ftns + self.observation_val]
        else:
            if not len(loss_weights) == len(self.ftns) + len(self.observation_val):
                raise ValueError(
                    f"Arguments `loss_weights` and `ftns + observation_val` must have same length."
                )

        for epoch in range(epochs):
            losses = []
            for i, pt in enumerate(self.pts):
                output = self.mlp(pt)
                losses.append(
                    loss_weights[i]
                    * self.loss(
                        self.ftns[i](pt, output, self.parameter),
                        torch.zeros_like(output),
                    )
                )
            for i, item in enumerate(self.observation_pts):
                output = self.mlp(item)
                losses.append(
                    loss_weights[i + len(self.pts)]
                    * self.loss(self.observation_val[i], output)
                )

            total_loss = sum(losses)
            self.optimizer.zero_grad()
            total_loss.backward()
            self.optimizer.step()

            self.loss_history.append(total_loss.item())
            self.param_history.append(
                list(map(lambda item: item.item(), self.parameter))
            )

            if epoch % 1000 == 0:
                print(
                    f"Epoch {epoch}/{epochs}, Loss: {total_loss:.6f}, Parameter: {self.parameter[0].item():.6f}"
                )

    def validation(self, exact_param: float):
        self.mlp.eval()

        # validation points
        x = torch.linspace(0, 1, 101).reshape(-1, 1)

        # exact solution
        y = torch.exp(exact_param * x)

        # evaluation
        with torch.no_grad():
            y_eval = self.mlp(x)

        # calculate mean relative L_2 norm
        error = torch.mean(torch.norm(y - y_eval, dim=1) / torch.norm(y, dim=1))
        param_error = (exact_param - self.parameter[0]) ** 2 / exact_param**2

        print(f"Validation Error: {error * 100:.4f} [%]")
        print(f"Parameter Error:  {param_error * 100:.4f} [%]")

        x = x.detach().cpu().numpy().flatten()
        y = y.detach().cpu().numpy().flatten()
        y_eval = y_eval.detach().cpu().numpy().flatten()

        train_x = torch.cat(self.pts).detach().cpu().numpy().flatten()
        train_y = np.exp(exact_param * train_x)

        observe_x = self.observation_pts[0].detach().cpu().numpy().flatten()
        observe_y = self.observation_val[0].detach().cpu().numpy().flatten()

        data = [
            go.Scatter(
                x=x,
                y=y,
                mode="lines",
                line=go.scatter.Line(width=5),
                name="True solution",
            ),
            go.Scatter(
                x=x,
                y=y_eval,
                mode="lines",
                line=go.scatter.Line(width=5, dash="dash"),
                name="Predicted solution",
            ),
            go.Scatter(
                x=train_x,
                y=train_y,
                mode="markers",
                marker=go.scatter.Marker(size=10),
                name="Training points",
            ),
            go.Scatter(
                x=observe_x,
                y=observe_y,
                mode="markers",
                marker=go.scatter.Marker(size=10),
                name="Observation points",
            ),
        ]
        layout = go.Layout(
            template="plotly_white",
            width=1300,
            height=1300,
            font=go.layout.Font(family="Times New Roman", size=25),
        )
        fig = go.Figure(data, layout)
        print(f"Results FIle Saved in {Path.cwd().absolute()}")
        pio.write_html(fig, Path.cwd() / "inverse_pinn_validation.html")

        data = [
            go.Scatter(
                y=self.loss_history,
                mode="lines",
                line=go.scatter.Line(width=5),
                name="loss",
            )
        ]
        fig = go.Figure(data, layout)
        pio.write_html(fig, Path.cwd() / "inverse_pinn_loss_history.html")

        param_history = np.asarray(self.param_history).T
        data = [
            go.Scatter(
                y=exact_param * np.ones(param_history.shape[1]),
                mode="lines",
                line=go.scatter.Line(width=5),
                name="Exact parameter",
            )
        ]
        for item in param_history:
            data.append(
                go.Scatter(
                    y=item.flatten(),
                    mode="lines",
                    line=go.scatter.Line(width=5),
                    name="Parameter",
                )
            )
        fig = go.Figure(data, layout)
        pio.write_html(fig, Path.cwd() / "inverse_pinn_param_history.html")


In [7]:
def derivative(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
        Calculate the derivative of `output` with respect to `input`.

        ...

        Parameters
        ----------
        input : torch.Tensor
            Input tensor of shape (batch_size, 1)
        output : torch.Tensor
            Output tensor of shape (batch_size, 1)

        Returns
        -------
        Derivative of `output` with respect to `input`.
    """

    return torch.autograd.grad(output, input, grad_outputs=torch.ones_like(output), create_graph=True)[0]

In [8]:
def differential_equation(input: torch.Tensor, output: torch.Tensor, params: List[torch.Tensor]) -> torch.Tensor:
    """
        Physics-Informed equation (y' = ay)

        ...

        Parameters
        ----------
        input : torch.Tensor
            Input tensor of shape (batch_size, 1)
        output : torch.Tensor
            Output tensor of shape (batch_size, 1)
        params: List[torch.Tensor]
            List of the parameters

        Returns
        -------
        Physics-Informed equation (y' = ay)
    """

    y = output
    dydx = derivative(input, output)

    return dydx - params[0] * y
#
#
#
def boundary_condition(input: torch.Tensor, output: torch.Tensor, params: List[torch.Tensor]) -> torch.Tensor:
    """
        Boundary condition (y(0) = 1)

        ...

        Parameters
        ----------
        input : torch.Tensor
            Input tensor of shape (the number of boundary points, 1)
        output : torch.Tensor
            Output tensor of shape (the number of boundary points, 1)
        params: List[torch.Tensor]
            List of the parameters

        Returns
        -------
        Boundary value at x = 0
    """

    y = output
    return y - torch.ones_like(y)

In [9]:
# exact parameter
a = 2.0

# Initial Parameter
param = torch.as_tensor(1.0)

# the number of points for training
n_points = 21

# Construct training points randomly in [0, 1]
x = torch.rand((n_points, 1)).requires_grad_(True)

# Boundary points (x = 0)
bp = torch.zeros((1, 1)).requires_grad_(True)

# the number of observation points
n_observation = 10

# Construct observation points in [0, 1]
observation_pts = torch.linspace(0, 1, n_observation).reshape(-1, 1)

# Observation value (exact solution at observation points)
observation_value = torch.exp(a * observation_pts)

# Differential equations and boundary value functions
ftns = [differential_equation, boundary_condition]

# Points list corresponding to ftns
pts = [x, bp]

pinn = IPINN(layer_size=[1, 5, 5, 1],activation=torch.nn.Tanh(),)
pinn.compile(
        ftns=list(ftns),
        pts=list(pts),
        observation_pts=[observation_pts],
        observation_val=[observation_value],
        parameter=[param],
    )

pinn.train(epochs=10000)
pinn.validation(exact_param=a)

Epoch 0/10000, Loss: 12.636535, Parameter: 0.999000
Epoch 1000/10000, Loss: 2.020401, Parameter: 0.935353
Epoch 2000/10000, Loss: 0.262644, Parameter: 1.649940
Epoch 3000/10000, Loss: 0.014428, Parameter: 1.954366
Epoch 4000/10000, Loss: 0.002927, Parameter: 2.006991
Epoch 5000/10000, Loss: 0.001501, Parameter: 2.010444
Epoch 6000/10000, Loss: 0.000815, Parameter: 2.010444
Epoch 7000/10000, Loss: 0.000469, Parameter: 2.009809
Epoch 8000/10000, Loss: 0.000298, Parameter: 2.008876
Epoch 9000/10000, Loss: 0.000208, Parameter: 2.007848
Validation Error: 0.1176 [%]
Parameter Error:  0.0012 [%]
Results FIle Saved in C:\Users\ADMIN\Dropbox\tha_files\NIMS\연구소 사업\부산영재교육원 RandE\2024년\학습자료\program\딥러닝\1. population mdoel
